# Plot up wind speed in NEMO forcing file

In [12]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import arrow
import cmocean

In [13]:
%matplotlib inline

## load data

In [14]:
ds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
ds

<xarray.Dataset>
Dimensions:    (gridX: 256, gridY: 266, time: 36312)
Coordinates:
  * time       (time) datetime64[ns] 2014-09-12 ... 2018-11-02T23:00:00
  * gridY      (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX      (gridX) float64 0.0 2.5e+03 5e+03 ... 6.35e+05 6.375e+05
Data variables:
    atmpres    (time, gridY, gridX) float32 ...
    precip     (time, gridY, gridX) float32 ...
    qair       (time, gridY, gridX) float32 ...
    solar      (time, gridY, gridX) float32 ...
    tair       (time, gridY, gridX) float32 ...
    therm_rad  (time, gridY, gridX) float32 ...
    u_wind     (time, gridY, gridX) float32 ...
    v_wind     (time, gridY, gridX) float32 ...
Attributes:
    acknowledgement:           MEOPAR, ONC, Compute Canada, Environment Canada
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    coverage_content_type:     modelResult
    creator_email:             sallen@eos.ubc.ca
    creator_name:

## Calculate wind speed.  

In [15]:
ds.u_wind

<xarray.DataArray 'u_wind' (time: 36312, gridY: 266, gridX: 256)>
[2472701952 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12 ... 2018-11-02T23:00:00
  * gridY    (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX    (gridX) float64 0.0 2.5e+03 5e+03 ... 6.325e+05 6.35e+05 6.375e+05
Attributes:
    _ChunkSizes:  [  1 266 256]
    level:        10 m above ground
    long_name:    U-Component of Wind
    short_name:   UGRD_10maboveground
    units:        m/s

### Figure out how to work with math.sqrt to calculate wind speed for 3D matrix

In [16]:
lt = 0
ly = 0
lx = 0
wind_spd = np.zeros((lt,ly,lx))
#wind_spd = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))

In [17]:
ds.u_wind.gridX

<xarray.DataArray 'gridX' (gridX: 256)>
array([      0.,    2500.,    5000., ...,  632500.,  635000.,  637500.])
Coordinates:
  * gridX    (gridX) float64 0.0 2.5e+03 5e+03 ... 6.325e+05 6.35e+05 6.375e+05
Attributes:
    _ChunkSizes:    256
    actual_range:   [      0.  637500.]
    long_name:      X
    standard_name:  x

In [18]:
getattr(ds.u_wind.gridX,'_ChunkSizes')

256

In [19]:
lt = getattr(ds.u_wind.time,'_ChunkSizes')
lx = getattr(ds.u_wind.gridX,'_ChunkSizes')
ly = getattr(ds.u_wind.gridY,'_ChunkSizes')

In [20]:
ly

266

In [21]:
lx

256

In [22]:
lt

1

###  Right.  Not what I want for "lt"....  Take two.

In [23]:
ds.u_wind.shape

(36312, 266, 256)

In [24]:
ds.u_wind.shape[1]

266

In [25]:
lt = ds.u_wind.shape[0]
ly = ds.u_wind.shape[1]
lx = ds.u_wind.shape[2]

In [26]:
lt

36312

In [27]:
ly

266

In [28]:
lx

256

###  That's better!!! :-) 

###  OK.  Let's try this AGAIN!  Can I to matrix sqrt or do I need to loop? First: Matrix square root

In [29]:
wind_spd = np.zeros((lt,ly,lx))
# wind_spd = np.array(np.sqrt(ds.u_wind**2 + ds.v_wind**2))

In [30]:
wind_spd[0,:,:] = np.array(math.sqrt(ds.u_wind[0,:,:]**2 + ds.v_wind[0,:,:]**2))

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
wind_spd[0,0,:] = np.array(math.sqrt(ds.u_wind[0,0,:]**2 + ds.v_wind[0,0,:]**2))

In [ ]:
wind_spd[0,0,0] = np.array(math.sqrt(ds.u_wind[0,0,0]**2 + ds.v_wind[0,0,0]**2))

### Well...for loops it is then

In [ ]:
test = [0,1,2,3]
for ix in test:
    print(test[ix])


###  Ah.  Wow.  I just learned that I went through all that effort to figure out the indexing of the matrix for NO GOOD REASON!!!!!

In [ ]:
for it in ds.u_wind:
    for iy in ds.u_wind:
        for ix in ds.u_wind:
            wind_spd[it,iy,ix] = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))
            

### Back to the drawing board!

In [ ]:
for it in range(0,3):
        wind_spd[it,0,0] = np.array(math.sqrt(ds.u_wind[it,0,0]**2 + ds.v_wind[it,0,0]**2))
        print(wind_spd[it,0,0])


In [ ]:
#for it in range(0,lt-1):
#    for iy in range(0,ly-1):
#        for ix in range(0,lx-1):
#            wind_spd[it,iy,ix] = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))
            

### working with numpy.square

In [ ]:
sq_wind_spd = np.zeros((lt,ly,lx))
sq_wind_spd[0,0,:] = ds.u_wind[0,0,:]**2 
sq_wind_spd = np.square(ds.u_wind) 